In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
import time
warnings.filterwarnings(action='ignore')

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website containing cocktails
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

# Wait for the page to load and for the cookie popup to disappear
wait = WebDriverWait(driver, 10)
wait.until(EC.invisibility_of_element_located((By.ID, 'onetrust-pc-btn-handler')))

# Initialize an empty dictionary to store cocktail names, links, and image links
cocktails_dict = {}

# Loop through each page from 1 to 22
for i in range(0, 22):
    # Get the URL of the page
    page_url = f"https://www.thecocktailproject.com/search-recipes/?page={i}"
    # Visit the page
    driver.get(page_url)
    
    # Wait for the cocktail elements to be present
    cocktail_elements = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.receipe-grid-items')))

    # Loop through each cocktail element on the current page
    for cocktail_element in cocktail_elements:
        # Extract name of the cocktail
        name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()

        # Extract link of the cocktail
        link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')

        # Extract image link of the cocktail
        image_element = cocktail_element.find_element(By.CSS_SELECTOR, '.field--name-field-recipe-image img')
        image_link = image_element.get_attribute('src')
        
        
        # Add the name, link, and image link of the cocktail to the dictionary
        cocktails_dict[name] = {'link': link, 'image_link': image_link}

# Close the browser
driver.quit()


In [5]:
cocktails_dict

{'Espresso Martini': {'link': 'https://www.thecocktailproject.com/Espresso-Martini',
  'image_link': 'https://www.thecocktailproject.com/sites/default/files/styles/grid-thumb/public/EspressoMartini.jpg?itok=161L5upy'},
 'French 75': {'link': 'https://www.thecocktailproject.com/French-75',
  'image_link': 'https://www.thecocktailproject.com/sites/default/files/styles/grid-thumb/public/French75.jpg?itok=hn6XEc65'},
 'Last Word': {'link': 'https://www.thecocktailproject.com/Last-Word',
  'image_link': 'https://www.thecocktailproject.com/sites/default/files/styles/grid-thumb/public/LastWord.jpg?itok=DFz5totk'},
 'Tom Collins': {'link': 'https://www.thecocktailproject.com/Tom-Collins',
  'image_link': 'https://www.thecocktailproject.com/sites/default/files/styles/grid-thumb/public/TomColins.jpg?itok=Vh6_KeE9'},
 'Tokyo Collins': {'link': 'https://www.thecocktailproject.com/Tokyo-Collins',
  'image_link': 'https://www.thecocktailproject.com/sites/default/files/styles/grid-thumb/public/Tokyo_

In [4]:
# 만들어뒀던 dict 로 이미지 다운로드, 이미지 이름 문제 있는 것들 renaming 해서 그것도 다시 새로운 dict image_names에 저장
import os
import requests

directory = "images_halways"
image_names = {}
if not os.path.exists(directory):
    os.makedirs(directory)
    
# Loop through each cocktail in the dictionary
for name, image_link in cocktails_dict.items():
    
    # Save image as PNG with cocktail name as file name
    response = requests.get(image_link)
    if response.status_code == 200:
        # Remove special characters from the name
       
        image_name = name
        image_name = image_name.replace("®", "")
        image_name = image_name.replace("\n","")
        image_names[name] = image_name
        
        # Define the file path
        file_path = f"images/{image_name}.png"
        
        # Write the new image to file
        with open(file_path, "wb") as f:
            f.write(response.content)


InvalidSchema: No connection adapters were found for "{'link': 'https://www.thecocktailproject.com/Espresso-Martini', 'image_link': 'https://www.thecocktailproject.com/sites/default/files/styles/grid-thumb/public/EspressoMartini.jpg?itok=161L5upy'}"

In [5]:
# 이미지 총 개수 440인거 확인
directory = "images"
files = os.listdir(directory)
num_files = len(files)

# Print the number of files
print(f"Number of files in {directory}: {num_files}")

Number of files in images: 440


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Create an empty dictionary to store ingredients for each cocktail
cocktails_with_ingredients = {}

for name, data in cocktails_dict.items():
    link = data['link']
    driver.get(link)

    # Wait for the page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'solid-block')))

    # Find all div elements with the class "title" within the solid block
    title_elements = driver.find_elements(By.CSS_SELECTOR, '.solid-block .title')

    # Extract ingredient names from the title elements and replace special characters
    #ingredient_names = [title.text.strip().replace('½', '1/2').replace('®', '').replace('¼', '1/4').replace('\n', ' ') for title in title_elements]
    # Extract ingredient names from the title elements and replace special characters
    ingredient_names = [title.text.strip().replace('½', '1/2').replace('®', '').replace('¼', '1/4').replace('\n', ' ') for title in title_elements if title.text.strip()]

    # Assign the ingredient list to the current cocktail
    data['ingredients'] = ingredient_names
    
    # Add the updated cocktail data to the new dictionary
    cocktails_with_ingredients[name] = data

# Update cocktails_dict with the new dictionary containing ingredient information
cocktails_dict.update(cocktails_with_ingredients)

# Close the browser
driver.quit()


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Create an empty dictionary to store ingredients for each cocktail
cocktails_with_description = {}

for name, data in cocktails_dict.items():
    link = data['link']
    driver.get(link)

    # Wait for the page to load

    try:
        description_element = driver.find_element(By.CLASS_NAME, 'recipe-copy')
        description_element = description_element.find_element(By.TAG_NAME, 'p')
        description = description_element.text.strip()
        print(description)
    except Exception:
        description = "NA"  # Set description to "NA" if the element is not found
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

    # Assign the description to the current cocktail
    data['description'] = description
    
    # Add the updated cocktail data to the new dictionary
    cocktails_with_description[name] = data

# Update cocktails_dict with the new dictionary containing ingredient information
cocktails_dict.update(cocktails_with_description)

# Close the browser
driver.quit()



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


















WebDriverException: Message: unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=122.0.6261.69)
Stacktrace:
	GetHandleVerifier [0x00E88CD3+51395]
	(No symbol) [0x00DF5F31]
	(No symbol) [0x00CAE004]
	(No symbol) [0x00C9EBD4]
	(No symbol) [0x00C9D98B]
	(No symbol) [0x00C9E1CD]
	(No symbol) [0x00CA920D]
	(No symbol) [0x00CB7101]
	(No symbol) [0x00CBA976]
	(No symbol) [0x00C9E5F8]
	(No symbol) [0x00CB6D66]
	(No symbol) [0x00D1A87B]
	(No symbol) [0x00D03C26]
	(No symbol) [0x00CDC629]
	(No symbol) [0x00CDD40D]
	GetHandleVerifier [0x01206493+3711107]
	GetHandleVerifier [0x0124587A+3970154]
	GetHandleVerifier [0x01240B68+3950424]
	GetHandleVerifier [0x00F39CD9+776393]
	(No symbol) [0x00E01704]
	(No symbol) [0x00DFC5E8]
	(No symbol) [0x00DFC799]
	(No symbol) [0x00DEDDC0]
	BaseThreadInitThunk [0x75567BA9+25]
	RtlInitializeExceptionChain [0x7723BD2B+107]
	RtlClearBits [0x7723BCAF+191]


In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

time.sleep(1)

# Find the birthday input fields and enter the date
month_input = driver.find_element(By.ID, 'edit-month-of-birth')
month_input.send_keys('08')

day_input = driver.find_element(By.ID, 'edit-day-of-birth')
day_input.send_keys('20')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('1999')
time.sleep(1)

app_submit = driver.find_element(By.CSS_SELECTOR, 'app-submit')
button_within_submit = app_submit.find_element(By.CSS_SELECTOR, 'button#edit-submit')
button_within_submit.click()
time.sleep(1)

reject_all_button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
reject_all_button.click()
time.sleep(1)

# Create an empty dictionary to store ingredients for each cocktail
cocktails_with_description = {}

for name, data in cocktails_dict.items():
    link = data['link']
    driver.get(link)
    time.sleep(3)
    try:
        aa = driver.find_element(By.CLASS_NAME, 'col-md-8')
        description_element = aa.find_element(By.TAG_NAME, 'p')
        description = description_element.text
        #print(description)
    except Exception:
        description = "NA"  # Set description to "NA" if the element is not found
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

    # Assign the description to the current cocktail
    data['description'] = description
    
    # Add the updated cocktail data to the new dictionary
    cocktails_with_description[name] = data

# Close the browser
driver.quit()

# Update cocktails_dict with the new dictionary containing ingredient information
cocktails_dict.update(cocktails_with_description)


Whether it's starting a great night or capping one off, this combination of EFFEN® Original Vodka, espresso and coffee liqueur is simple to make before, after or right in the middle of any fun gathering.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Historically found in the speakeasy, this distinguished cocktail can now be found right in your home. Part Sipsmith® London Dry Gin, part evening standout, this well-chilled coupe glass dweller keeps every evening classy.
Not everyone agrees where the name Tom Collins comes from, but they do agree it's a great cocktail to enjoy after brunch. A combination of Sipsmith® London Dry Gin, lemon juice, club soda and simple syrup, this bright and shining social sparkler will fit right in on any park blanket or porch table.
If you ever want to visit Tokyo, try starting at home. Made with Suntory® Roku Gin, this light, elegant and simple cocktail travels well from kitchen to coffee table and beyond.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [44]:
import csv

# Define the file name for the CSV file
csv_file = 'cocktails_halways.csv'

# Define the field names for the CSV file
field_names = ['Cocktail Name', 'Link', 'Image Link', 'Ingredients', 'Description']

# Add description to cocktails_dict
for cocktail_name, cocktail_info in cocktails_dict.items():
    # Add description if available, otherwise use an empty string
    description = cocktail_info.get('description', '')
    cocktail_info['description'] = description

# Write the cocktails_dict to the CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=field_names)
    
    # Write the header row
    writer.writeheader()
    
    # Write each cocktail entry
    for cocktail_name, cocktail_info in cocktails_dict.items():
        # Replace ® symbol with empty string in cocktail name
        cocktail_name = cocktail_name.replace('®', '')
        
        # Check if the 'ingredients' key exists
        if 'ingredients' in cocktail_info:
            # If the key exists, write the ingredients as a comma-separated string
            writer.writerow({
                'Cocktail Name': cocktail_name,
                'Link': cocktail_info['link'],
                'Image Link': cocktail_info['image_link'],
                'Ingredients': ', '.join(cocktail_info['ingredients']),
                'Description': cocktail_info['description']
            })
        else:
            # If the key does not exist, write an empty string for ingredients
            writer.writerow({
                'Cocktail Name': cocktail_name,
                'Link': cocktail_info['link'],
                'Image Link': cocktail_info['image_link'],
                'Ingredients': '',
                'Description': cocktail_info['description']
            })

print("CSV file created successfully!")


CSV file created successfully!


In [45]:
# Initialize a variable to count the number of 'NA' descriptions
na_description_count = 0

# Iterate over the cocktails dictionary
for cocktail_info in cocktails_dict.values():
    # Check if the description is 'NA'
    if cocktail_info.get('description') == 'NA':
        # Increment the count if the description is 'NA'
        na_description_count += 1

print("Number of cocktails with 'NA' description:", na_description_count)


Number of cocktails with 'NA' description: 211


In [52]:
import csv

# Function to filter out unwanted ingredients
def filter_ingredients(ingredient_list):
    # Words to check for in ingredient names
    unwanted_words = ['How', 'When', 'What', 'Do', 'Where', 'Why', 'when', 'how', 'why', 'what', 'Cocktails', 'Cocktail', 'Home', 'home', 'Guide', 'Your', 'Bar', 'Party', 'Is', 'Should', 'vs.', 'The']
    # Filter out ingredients that contain unwanted words
    filtered_ingredients = [ingredient.strip() for ingredient in ingredient_list if not any(word in ingredient for word in unwanted_words)]
    return filtered_ingredients

# Define the file names for the input and output CSV files
input_csv_file = 'cocktails_halways.csv'
output_csv_file = 'final.csv'

# Read the CSV file and filter ingredients
filtered_cocktails_dict = {}
with open(input_csv_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        cocktail_name = row['Cocktail Name']
        ingredients = row['Ingredients'].split(', ')
        filtered_ingredients = filter_ingredients(ingredients)
        filtered_cocktails_dict[cocktail_name] = filtered_ingredients

# Write the filtered data to a new CSV file
with open(output_csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Name', 'Ingredients'])
    writer.writeheader()
    for cocktail_name, ingredients in filtered_cocktails_dict.items():
        writer.writerow({'Name': cocktail_name, 'Ingredients': ', '.join(ingredients)})

print("Filtered cocktails saved to:", output_csv_file)


Filtered cocktails saved to: final.csv


In [53]:
import pandas as pd

# # Load the first CSV file
# df1 = pd.read_csv('cocktails2.csv')
# df2 = pd.read_csv('final.csv')

# # Load the first CSV file with specified encoding
df1 = pd.read_csv('cocktails2.csv', encoding='ISO-8859-1')
df2 = pd.read_csv('final.csv', encoding='ISO-8859-1')


merged_df = pd.merge(df1, df2, on='Name')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)

print("CSV files merged successfully!")

CSV files merged successfully!


In [56]:
df1 = pd.read_csv('merged_file.csv', encoding='ISO-8859-1')
df2 = pd.read_csv('merged.csv', encoding='ISO-8859-1')


merged_df = pd.merge(df1, df2, on='Name')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('fianl data.csv', index=False)

print("FINAL CSV files merged successfully!")

FINAL CSV files merged successfully!
